# Begin 2nd Iteration

In [1]:
import wrangle as w
import model as m
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

## Model.py (customized)

In [2]:
import pandas as pd
import numpy as np
import wrangle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

#-----------------------------------------------------------------------------

# Model Prep


def dummy_dept(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['dept'])
    # Name the new columns
    dummy_df.columns = ['animal_care_services', 'code_enforcement_services', 
                        'customer_services', 'development_services', 
                        'metro_health', 'parks_and_rec',
                        'solid_waste_management', 'trans_and_cap_improvements', 
                        'unknown_dept']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------    
def dummy_call_reason(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['call_reason'])
    # Name the new columns
    dummy_df.columns = ['buildings', 'business', 'cleanup', 'code',
                        'customer_service', 'field', 'land',
                        'license', 'misc', 'storm', 'streets', 'trades', 
                        'traffic', 'waste']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------
def make_source_id_dummies(df):
    '''This function takes in the cleaned dataframe, makes dummy variables of the source id column, readds the names of the
    dummy columns and returns the concatenated dummy dataframe to the original dataframe.'''
    #make dummies
    dummy_df = pd.get_dummies(df['source_id'])
    #add back column names
    dummy_df.columns = ['web_portal', '311_mobile_app', 'constituent_call', 'internal_services_requests']
    # concatenate dummies to the cleaned data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df

#-------------------------------
#zipcode dummies
def dummy_zipcodes(df):
    dummy = pd.get_dummies(df['zipcode'])
    df = pd.concat([df, dummy], axis=1)
    return df

#-------------------------------
def keep_info(df):
    df.drop(df.columns.difference(['dept','call_reason', 'source_id', 'level_of_delay',
                                   'council_district', 'resolution_days_due', 'district_0', 'district_1', 'district_2',
                                   'district_3', 'district_4','district_5', 'district_6', 'district_7', 'district_8', 
                                   'district_9','district_10', 'per_capita_income', 'zipcode']), 1, inplace=True)
    return df

#--------------------------------
def model_df():
    '''This function reads in the clean 311 dataframe, applies all of the above functions to prepare it for modeling. 
    The function then returns a cleaned dataframe ready for modeling.'''
    df= wrangle.clean_311(wrangle.get_311_data())
    df= keep_info(df)
    df= dummy_dept(df)
    df= dummy_call_reason(df)
    df= make_source_id_dummies(df)
    df= dummy_zipcodes(df)

    return df
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def split(df, stratify_by= 'level_of_delay'):
    """
    Crude train, validate, test split
    To stratify, send in a column name
    """
    if stratify_by == None:
        train, test = train_test_split(df, test_size=.2, random_state=319)
        train, validate = train_test_split(train, test_size=.3, random_state=319)
    else:
        train, test = train_test_split(df, test_size=.2, random_state=319, stratify=df[stratify_by])
        train, validate = train_test_split(train, test_size=.3, random_state=319, stratify=train[stratify_by])
    return train, validate, test


#------------------------------------
def separate_y(train, validate, test):
    '''
    This function will take the train, validate, and test dataframes and separate the target variable into its
    own panda series
    '''
    
    X_train = train.drop(columns=['level_of_delay'])
    y_train = train.level_of_delay
    X_validate = validate.drop(columns=['level_of_delay'])
    y_validate = validate.level_of_delay
    X_test = test.drop(columns=['level_of_delay'])
    y_test = test.level_of_delay
    return X_train, y_train, X_validate, y_validate, X_test, y_test
#------------------------------------
def scale_data(X_train, X_validate, X_test):
    '''
    This function will scale numeric data using Min Max transform after 
    it has already been split into train, validate, and test.
    '''
    
    
    obj_col = []
    num_train = X_train.drop(columns = obj_col)
    num_validate = X_validate.drop(columns = obj_col)
    num_test = X_test.drop(columns = obj_col)
    
    
    # Make the thing
    scaler = preprocessing.MinMaxScaler()
    
   
    # we only .fit on the training data
    scaler.fit(num_train)
    train_scaled = scaler.transform(num_train)
    validate_scaled = scaler.transform(num_validate)
    test_scaled = scaler.transform(num_test)
    
    # turn the numpy arrays into dataframes
    train_scaled = pd.DataFrame(train_scaled, columns=num_train.columns)
    validate_scaled = pd.DataFrame(validate_scaled, columns=num_train.columns)
    test_scaled = pd.DataFrame(test_scaled, columns=num_train.columns)
    
    
    return train_scaled, validate_scaled, test_scaled

#------------------------------------

def split_separate_scale(df, stratify_by= 'level_of_delay'):
    '''
    This function will take in a dataframe
    separate the dataframe into train, validate, and test dataframes
    separate the target variable from train, validate and test
    then it will scale the numeric variables in train, validate, and test
    finally it will return all dataframes individually
    '''
    
    # split data into train, validate, test
    train, validate, test = split(df, stratify_by= 'level_of_delay')
    
     # seperate target variable
    X_train, y_train, X_validate, y_validate, X_test, y_test = separate_y(train, validate, test)
    
    
    # scale numeric variable
    train_scaled, validate_scaled, test_scaled = scale_data(X_train, X_validate, X_test)
    
    return train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled

In [3]:
df = w.clean_311(w.get_311_data())

/Users/lorisegovia/codeup-data-science/project/workbooks/lori/wrangle.py:331: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['open_week'] = df.open_date.dt.week


In [4]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
df.isna().sum()

open_date                       0
due_date                        0
closed_date                 43182
is_late                         0
dept                            0
call_reason                     0
case_type                       0
case_status                     0
source_id                       0
address                         0
council_district                0
longitude                       0
latitude                        0
days_open                       0
resolution_days_due             0
days_before_or_after_due    43182
pct_time_of_used                0
level_of_delay                  0
district_1                      0
district_2                      0
district_3                      0
district_4                      0
district_5                      0
district_6                      0
district_7                      0
district_8                      0
district_9                      0
district_10                     0
voter_turnout_2019              0
num_of_registe

In [6]:
df.shape

(399986, 36)

In [7]:
df.dropna(subset=['days_before_or_after_due'], how='all', inplace=True)

In [8]:
df.dropna(subset=['closed_date'], how='all', inplace=True)

In [9]:
df.isna().sum()

open_date                   0
due_date                    0
closed_date                 0
is_late                     0
dept                        0
call_reason                 0
case_type                   0
case_status                 0
source_id                   0
address                     0
council_district            0
longitude                   0
latitude                    0
days_open                   0
resolution_days_due         0
days_before_or_after_due    0
pct_time_of_used            0
level_of_delay              0
district_1                  0
district_2                  0
district_3                  0
district_4                  0
district_5                  0
district_6                  0
district_7                  0
district_8                  0
district_9                  0
district_10                 0
voter_turnout_2019          0
num_of_registered_voters    0
zipcode                     0
open_month                  0
open_year                   0
open_week 

In [10]:
df.shape

(356804, 36)

In [11]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,...,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",...,0,0,0.086,67656,78203,1,2017,4,19055,59.81
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",...,0,0,0.078,66370,78242,1,2017,4,18500,65.21
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",...,0,0,0.086,67656,78202,2,2017,5,19055,59.81
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",...,0,0,0.148,68081,78207,2,2017,5,23967,26.00
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",...,0,0,0.124,80007,78251,2,2017,9,23437,38.44


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356804 entries, 551 to 399979
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   open_date                 356804 non-null  datetime64[ns]
 1   due_date                  356804 non-null  datetime64[ns]
 2   closed_date               356804 non-null  datetime64[ns]
 3   is_late                   356804 non-null  object        
 4   dept                      356804 non-null  object        
 5   call_reason               356804 non-null  object        
 6   case_type                 356804 non-null  object        
 7   case_status               356804 non-null  object        
 8   source_id                 356804 non-null  object        
 9   address                   356804 non-null  object        
 10  council_district          356804 non-null  int64         
 11  longitude                 356804 non-null  float64       
 12  

In [ ]:
#df.to_csv('cleanest_311.csv')

In [ ]:
#zipcode dummies
#def dummy_zipcodes(df):
    #dummy = pd.get_dummies(df['zipcode'])
    #df = pd.concat([df, dummy], axis=1)
    #return df

In [ ]:
#dummy = pd.get_dummies(df['zipcode'])
#dummy.head()

In [ ]:
#df = pd.concat([df, dummy], axis=1)
#df.head()

In [13]:
df = model_df()

/Users/lorisegovia/codeup-data-science/project/workbooks/lori/wrangle.py:331: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['open_week'] = df.open_date.dt.week


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399986 entries, 0 to 399985
Columns: 112 entries, dept to 78288
dtypes: category(1), int64(3), object(4), uint8(104)
memory usage: 64.5+ MB


In [15]:
df = df.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district'])

In [16]:
df = df.drop(columns = ['zipcode'])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399986 entries, 0 to 399985
Columns: 107 entries, resolution_days_due to 78288
dtypes: category(1), int64(2), uint8(104)
memory usage: 49.2 MB


In [19]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = m.split_separate_scale(df)

### baseline

In [20]:
#baseline
train.level_of_delay.value_counts()

Extremely Early Response    142657
Very Early Response          35776
Late Response                17032
Early Response               13609
On Time Response             11761
Very Late Response            2510
Extremely Late Response        646
Name: level_of_delay, dtype: int64

In [21]:
#baseline accuracy will be early response
baseline = round((train.level_of_delay == 'Extremely Early Response').mean(), 2) *100

print(f'The baseline accuracy is: {baseline} %')

The baseline accuracy is: 64.0 %


In [29]:
train.corr()

,resolution_days_due,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,per_capita_income,animal_care_services,code_enforcement_services,customer_services,development_services,metro_health,parks_and_rec,solid_waste_management,trans_and_cap_improvements,unknown_dept,buildings,business,cleanup,code,customer_service,field,land,license,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests,78015,78023,78073,78109,78112,78154,78201,78202,78203,78204,78205,78207,78208,78209,78210,78211,78212,78213,78214,78215,78216,78217,78218,78219,78220,78221,78222,78223,78224,78225,78226,78227,78228,78229,78230,78231,78232,78233,78234,78235,78236,78237,78238,78239,78240,78242,78244,78245,78247,78248,78249,78250,78251,78252,78253,78254,78255,78256,78257,78258,78259,78260,78261,78264,78266,78284,78288
resolution_days_due,1.000000,0.049681,0.039010,0.003409,-0.037011,0.025480,-0.019688,-0.008060,-0.023338,-0.043250,-0.023055,-0.052886,-0.218074,0.111491,-0.046796,0.726757,-0.047110,0.003828,-0.526424,0.027994,0.009130,0.055049,-0.029773,0.014547,0.731070,-0.057732,-0.218266,0.047319,-0.025982,-0.157496,0.050130,0.019132,0.012070,-0.010121,-0.459794,0.434040,0.021818,0.094915,-0.383693,NaN,-0.020775,-0.000656,-0.023957,0.014699,0.007876,0.035533,0.011764,0.018376,0.015940,0.010085,0.021532,0.016628,0.005166,0.020583,-0.008530,0.017956,0.014067,0.020323,0.010619,0.028191,0.011304,0.007919,0.022342,0.017247,-0.015786,-0.002760,-0.004469,-0.020853,0.012388,0.004803,-0.000173,-0.003358,0.021788,-0.020581,-0.010685,-0.020828,-0.003384,0.011578,-0.007935,-0.002665,-0.009168,0.003615,-0.008752,-0.007363,-0.015730,0.005894,-0.016026,-0.028608,-0.025949,-0.020700,-0.030993,-0.010419,-0.017606,0.003087,-0.018524,-0.006684,-0.004168,0.002511,-0.024504,-0.035211,-0.003883,-0.001636,0.009448,-0.011388,0.002461,0.000351
district_1,0.049681,1.000000,-0.152768,-0.148012,-0.126912,-0.157534,-0.120253,-0.126808,-0.092692,-0.091495,-0.115487,0.062027,-0.038242,-0.007648,0.002906,0.077876,0.006471,0.003088,-0.065254,0.014817,0.037125,0.022889,0.007223,-0.000050,0.075209,0.033750,-0.038349,-0.000473,0.004710,-0.009915,-0.000520,0.017675,0.006157,0.015464,-0.061867,0.042277,0.079879,-0.005824,-0.086647,NaN,-0.021498,-0.008174,-0.022105,-0.007114,-0.002173,0.398040,-0.053585,-0.039226,0.006055,0.173218,0.064660,-0.028059,-0.000775,-0.019694,-0.067238,0.446531,0.301076,-0.061276,0.077336,0.220763,-0.052509,-0.061898,-0.031573,-0.058628,-0.068745,-0.052303,-0.075492,-0.045672,-0.046238,-0.031928,-0.075895,-0.096355,-0.035954,0.004667,-0.023214,-0.047049,-0.055798,-0.006092,-0.011326,-0.001643,-0.080664,-0.042752,-0.020330,-0.051562,-0.063147,-0.035753,-0.050840,-0.065216,-0.027470,-0.059263,-0.068326,-0.062127,-0.022771,-0.009859,-0.042825,-0.017285,-0.017957,-0.014236,-0.034295,-0.036381,-0.009074,-0.001162,-0.008498,-0.010650,-0.001643,-0.002597
district_2,0.039010,-0.152768,1.000000,-0.149642,-0.128309,-0.159268,-0.121578,-0.128204,-0.093713,-0.092503,-0.116758,-0.204983,0.003455,0.025016,-0.000428,0.040371,-0.010755,0.004830,-0.033373,-0.021308,-0.005554,0.015395,-0.003139,-0.001494,0.043901,-0.006111,0.003151,0.003447,-0.006844,-0.019661,-0.014101,-0.010571,0.010070,-0.020279,-0.025260,0.083294,-0.033939,0.017239,-0.047665,NaN,-0.021735,-0.008264,0.144697,-0.007193,0.002495,-0.088252,0.350760,0.258232,-0.050271,-0.011997,-0.097504,0.183486,0.077823,0.086485,-0.067978,-0.052199,-0.063237,-0.061951,0.039032,-0.059934,-0.019081,0.381446,0.204397,0.383775,-0.069502,0.152336,-0.076323,-0.046175,-0.046747,-0.032280,-0.076731,-0.097416,-0.036350,-0.052458,-0.023470,-0.047567,-0.031702,0.036529,-0.011451,-0.001661,-0.081552,-0.043223,0.051862,-0.052130,-0.063842,0.234036,-0.051400,-0.065934,-0.027772,-0.059916,-0.069079,-0.062811,-0.023022,-0.009968,-0.043296,-0.017475,-0.018155,-0.014392,-0.034673,-0.036782,-0.009174,-0.001174,-0.008592,-0.010767,-0.001661,-0.0026

In [31]:
train.corr().iloc[15].sort_values(ascending=False)[0:10]

development_services    1.000000
code                    0.969370
resolution_days_due     0.726757
web_portal              0.579270
district_1              0.077876
311_mobile_app          0.070828
78201                   0.055494
district_5              0.043625
78207                   0.041537
district_2              0.040371
Name: development_services, dtype: float64

### DT

In [ ]:
#make the thing
clf = DecisionTreeClassifier(max_depth=3, random_state=123)
#fit the thing
clf = clf.fit(X_train, y_train)
#predicitons
y_pred = clf.predict(X_train)
#probability
y_pred_proba = clf.predict_proba(X_train)

In [ ]:
#compute the accuracy 
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

In [ ]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

In [ ]:
# Produce y_predictions that come from the X_validate
y_pred = clf.predict(X_validate)

# Compare actual y values (from validate) to predicted y_values from the model run on X_validate
print(classification_report(y_validate, y_pred))

### DT 2

In [ ]:
#make the thing
clf = DecisionTreeClassifier(max_depth=6, random_state=123)
#fit the thing
clf = clf.fit(X_train, y_train)
#predicitons
y_pred = clf.predict(X_train)
#probability
y_pred_proba = clf.predict_proba(X_train)

In [ ]:
#compute the accuracy 
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

In [ ]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

### DT 3

In [ ]:
#make the thing
clf = DecisionTreeClassifier(max_depth=12, random_state=123)
#fit the thing
clf = clf.fit(X_train, y_train)
#predicitons
y_pred = clf.predict(X_train)
#probability
y_pred_proba = clf.predict_proba(X_train)

In [ ]:
#compute the accuracy 
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

In [ ]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

### Vanilla KNN

In [ ]:
train.columns.to_list()

In [ ]:
features1 = ['resolution_days_due',
 'district_1',
 'district_2',
 'district_3',
 'district_4',
 'district_5',
 'district_6',
 'district_7',
 'district_8',
 'district_9',
 'district_10',
 'per_capita_income',
 'animal_care_services',
 'code_enforcement_services',
 'customer_services',
 'development_services',
 'metro_health',
 'parks_and_rec',
 'solid_waste_management',
 'trans_and_cap_improvements',
 'unknown_dept',
 'buildings',
 'business',
 'cleanup',
 'code',
 'customer_service',
 'field',
 'land',
 'license',
 'misc',
 'storm',
 'streets',
 'trades',
 'traffic',
 'waste',
 'web_portal',
 '311_mobile_app',
 'constituent_call',
 'internal_services_requests',
 '78015',
 '78023',
 '78073',
 '78109',
 '78112',
 '78154',
 '78201',
 '78202',
 '78203',
 '78204',
 '78205',
 '78207',
 '78208',
 '78209',
 '78210',
 '78211',
 '78212',
 '78213',
 '78214',
 '78215',
 '78216',
 '78217',
 '78218',
 '78219',
 '78220',
 '78221',
 '78222',
 '78223',
 '78224',
 '78225',
 '78226',
 '78227',
 '78228',
 '78229',
 '78230',
 '78231',
 '78232',
 '78233',
 '78234',
 '78235',
 '78236',
 '78237',
 '78238',
 '78239',
 '78240',
 '78242',
 '78244',
 '78245',
 '78247',
 '78248',
 '78249',
 '78250',
 '78251',
 '78252',
 '78253',
 '78254',
 '78255',
 '78256',
 '78257',
 '78258',
 '78259',
 '78260',
 '78261',
 '78264',
 '78266',
 '78284',
 '78288']
#weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=10, weights='uniform')
#fit model
knn.fit(X_train[features1], y_train)

In [ ]:
#mkae predictions
y_pred_knn = knn.predict(X_train[features1])
#estimate probability
y_pred_proba = knn.predict_proba(X_train[features1])

In [ ]:
#accuracy
accuracy = knn.score(X_train[features1], y_train)
print(f"KNN Accuracy is {accuracy:.5}")

## Feature engineering

In [23]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
train_scaled.head()

In [ ]:
train_scaled.info()

In [ ]:
y_train.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 8 features
f_selector = SelectKBest(f_regression, k=8)

# find the top 8 X's correlated with y
f_selector.fit(train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = train_scaled.iloc[:,feature_mask].columns.tolist()

In [ ]:
f_feature

### 2nd round of modelz

In [33]:
features1 = ['code', 
'resolution_days_due',  
'web_portal', 
'district_1', 
'311_mobile_app', 
'78201', 
'district_5', 
'78207', 
'district_2'] 
#weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=6, weights='uniform')
#fit model
knn.fit(X_train[features1], y_train)

KNeighborsClassifier(n_neighbors=6)

In [34]:
#mkae predictions
y_pred_knn = knn.predict(X_train[features1])
#estimate probability
y_pred_proba = knn.predict_proba(X_train[features1])

In [35]:
#accuracy
accuracy = knn.score(X_train[features1], y_train)
print(f"KNN Accuracy is {accuracy:.5}")

KNN Accuracy is 0.56181
